In [237]:
import kagglehub
saurabh00007_iriscsv_path = kagglehub.dataset_download('saurabh00007/iriscsv')

print('Data source import complete.')

Data source import complete.


# Import Libraries

In [238]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import plotly.express as px
import plotly.graph_objects as go

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/iriscsv/Iris.csv


# Import DataSet

In [239]:
df = pd.read_csv("/kaggle/input/iriscsv/Iris.csv")
df.head(10)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


# Data Understanding

In [240]:
df.shape

(150, 6)

In [241]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [242]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [243]:
df.duplicated().sum()

np.int64(0)

 There are no duplicate rows.

In [244]:
df.isna().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


There are no missing values in any of the columns.

# EDA

In [245]:
df['Species'].value_counts()

,count
Species,
Iris-setosa,50
Iris-versicolor,50
Iris-virginica,50


In [246]:
data = {'Species': ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],
        'count': [50, 50, 50]}
df_species_counts = pd.DataFrame(data)

fig = px.pie(
    df_species_counts,
    values='count',
    names='Species',
    title='Distribution of Iris Flower Species',
    hole=0.3,
    width=600,
    height=400
)

fig.show()

The dataset is perfectly balanced, with each of the three species:
Iris-setosa, Iris-versicolor, and Iris-virginica—representing an equal proportion of the total. Each species accounts for exactly 33.3% of the samples. This balanced distribution is ideal for training a classification model, as it prevents the model from being biased towards any single species.

In [247]:
fig_violin_sepal_length = px.violin(
    df,
    x="Species",
    y="SepalLengthCm",
    color="Species",
    box=True,
    points="all",
    title="Distribution of Sepal Length by Species",
    width=1000,
    height=500
)
fig_violin_sepal_length.show()

Iris-setosa has the shortest sepal length, with its median around 5.0 cm.

Iris-versicolor has a medium sepal length, with its median around 5.9 cm.

Iris-virginica has the longest sepal length, with its median around 6.5 cm.

sepal length is a good feature to distinguish Iris-setosa from the other two species, as its length is significantly shorter. However, there is a noticeable overlap in sepal length between Iris-versicolor and Iris-virginica, meaning sepal length alone is not sufficient to perfectly separate these two species.

In [248]:
fig_violin_sepal_Width = px.violin(
    df,
    x="Species",
    y="SepalWidthCm",
    color="Species",
    box=True,
    points="all",
    title="Distribution of Sepal Width by Species",
    width=1000,
    height=500
)
fig_violin_sepal_Width.show()

Sepal width is a highly effective feature for distinguishing Iris-setosa from the other two species. The Iris-setosa distribution is centered on a distinctly wider sepal width and is well-separated from the others.

However, sepal width is a poor feature for separating Iris-versicolor and Iris-virginica. The distributions for these two species have a significant overlap and are very similar in shape and location, making it difficult to classify them based on this measurement alone.

In [249]:
fig_petal_length = px.box(
    df,
    x="Species",
    y="PetalLengthCm",
    color="Species",
    title="Distribution of Petal Length by Species",
    width=1000,
    height=500
)
fig_petal_length.show()

Iris-setosa has a distinctly short petal length with its median around 1.5 cm,
making it easily separable from the other two.

Iris-versicolor has a medium petal length that is clearly separated from both Iris-setosa and Iris-virginica and with median around 4.35 cm.

Iris-virginica has the longest petal length with median around 5.1, with its distribution not overlapping with the others.

This visualization suggests that a simple rule based on petal length alone could achieve a very high level of accuracy in classifying all three species.


In [250]:
fig_petal_Width = px.box(
    df,
    x="Species",
    y="PetalWidthCm",
    color="Species",
    title="Distribution of Petal Width by Species",
    width=1000,
    height=500
)
fig_petal_Width.show()

Iris-setosa has a very narrow petal width, with its distribution well below the others.

Iris-versicolor has a medium petal width, and its range is clearly separate from both Iris-setosa and Iris-virginica.

Iris-virginica has the widest petal width, and its distribution is distinct from the other two.

This visualization strongly suggests that petal width is an excellent feature for building a highly accurate classification model.

# Data Preprocessing

## Change categorical data into numerical data

In [251]:
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
categorical_cols = df.select_dtypes(exclude=['number']).columns.tolist()

numerical_cols: A list of the names of all columns that contain numbers (like integers or floats).

categorical_cols: A list of the names of all columns that do not contain numbers, which are typically considered categorical (like text or boolean values).

In [252]:
print("Numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

Numerical Columns: ['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
Categorical Columns: ['Species']


"Species" column is the only categorical column

### by using LabelEncoder

In [253]:
le = LabelEncoder()
df["Species"] = le.fit_transform(df["Species"])

In [254]:
df['Species'].value_counts()

,count
Species,
0,50
1,50
2,50


The number 0 represents the first category of flower species.

The number 1 represents the second category.

The number 2 represents the third category.

In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 7.2 KB


Now all columns are numerical

## Drop columns

In [256]:
df.drop('Id', axis=1, inplace=True)

The "Id" column was dropped because it serves as a unique identifier for each data point and does not contain any meaningful information that would help a machine learning model classify the flower species.

## Split the Dataset

In [257]:
X = df.drop("Species", axis=1)
y = df["Species"]

Splitting the data into "features" (X) and the "target" (y) to prepare for a machine learning model.

In [258]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Splitting the data into training and testing sets to ensure the model can generalize to new, unseen data.

Allocates 20% of the data for testing and 80% for training

## Scaling the dataset by using StandardScaler

In [259]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Standardizing the numerical features to ensure they are all on the same scale, which prevents some features from dominating the model simply due to their larger values.

# Data Modeling

In [260]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

 Selecting a K-Nearest Neighbors model and training it on the preprocessed data to begin the learning process.

 With number of neighbors = 3

In [261]:
y_pred = model.predict(X_test_scaled)

Using the trained model to predict the class (species) of the flowers based on the

In [262]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accuracy:.4f}")

Accuracy = 1.0000


The model achieved a perfect, 100% accuracy in predicting the species of every single flower in the test set.

In [263]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



The model achieved a perfect 100% performance on all evaluation metrics, meaning it made no errors in classifying any of the flower species.

Perfect Precision (Precision = 1.00): The model made no incorrect positive predictions. Every time it predicted a specific class, it was right.

Perfect Recall (Recall = 1.00): The model did not miss a single case. It successfully identified all the flowers of each type in the test set.

Perfect F1-Score (F1-Score = 1.00): This confirms the flawless performance across all levels, as it is the harmonic mean of precision and recall.

Perfect Overall Accuracy (Accuracy = 1.00): This confirms that the predictions were 100% correct for all 30 samples in the test set.



In [264]:
print("\n--- Evaluation Metrics ---")
print("Confusion Matrix:")
cm= confusion_matrix(y_test, y_pred)
print(cm)


--- Evaluation Metrics ---
Confusion Matrix:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


In [265]:
class_names = le.classes_

annotations = []
for i, row in enumerate(cm):
    for j, value in enumerate(row):
        color = 'white' if value > 0 else 'black'
        annotations.append(
            dict(
                x=class_names[j],
                y=class_names[i],
                text=str(value),
                font=dict(color=color),
                showarrow=False
            )
        )

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=class_names,
    y=class_names,
    colorscale='Blues'
))

fig.update_layout(
    title='Confusion Matrix for Iris Flower Classification',
    xaxis_title='Predicted Labels',
    yaxis_title='Actual Labels',
    annotations=annotations
)

fig.show()

The model achieved perfect performance, making zero classification errors for any of the flower species, which visually confirms the 100% accuracy.

The presence of numbers (10, 9, 11) only on this diagonal means the model predicted everything correctly.

The zeros everywhere else in the matrix confirm that the model made no incorrect predictions whatsoever.